In [80]:
# Modèle théorique d'estimation des émissions sur le périphérique parisien
# Inspiré de la décomposition de Bigo (2020)

# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider



In [81]:
# --- Paramètres par défaut ---
def default_params():
    return {
        "distance_moyenne_km": 35,
        "nombre_vehicules": 100_000,
        "part_voitures_thermiques": 0.7,
        "part_voitures_diesel": 0.2,
        "part_voitures_elec": 0.1,
        "conso_l_100km_essence": 7.0,
        "conso_l_100km_diesel": 5.5,
        "conso_kWh_100km_elec": 17.0,
        "emission_CO2_l_essence": 2.3,
        "emission_CO2_l_diesel": 2.6,
        "emission_CO2_kWh": 0.012,
        "emission_NOx_g_km_essence": 0.25,
        "emission_NOx_g_km_diesel": 0.6,
        "emission_NOx_g_km_elec": 0.01,
        "emission_PM10_g_km_essence": 0.01,
        "emission_PM10_g_km_diesel": 0.05,
        "emission_PM10_g_km_elec": 0.005,
        "taux_remplissage_actuel": 1.2,
        "taux_remplissage_covoit": 1.8,
        "part_covoiturage": 0.2,
        "surconso_congestion_pointe": 1.1,
        "surconso_congestion_offpointe": 1.0,
        "duree_jours": 365,
        "part_temps_pointe": 0.31
    }

In [82]:
# --- Fonction d'estimation ---
def calcul_emissions(params, vitesse_reduite=False, part_temps=1.0, surconso_congestion=1.0):
    d = params["distance_moyenne_km"]
    n = params["nombre_vehicules"]
    jours = params["duree_jours"]

    covoit = params["part_covoiturage"]
    r = (1 - covoit) * params["taux_remplissage_actuel"] + covoit * params["taux_remplissage_covoit"]
    demande_eq = n * d / r

    result = {"CO2": 0, "NOx": 0, "PM10": 0}

    for type_v, part, conso, CO2e, NOx, PM in [
        ("essence", params["part_voitures_thermiques"], params["conso_l_100km_essence"], params["emission_CO2_l_essence"], params["emission_NOx_g_km_essence"], params["emission_PM10_g_km_essence"]),
        ("diesel", params["part_voitures_diesel"], params["conso_l_100km_diesel"], params["emission_CO2_l_diesel"], params["emission_NOx_g_km_diesel"], params["emission_PM10_g_km_diesel"]),
        ("elec", params["part_voitures_elec"], params["conso_kWh_100km_elec"], params["emission_CO2_kWh"], params["emission_NOx_g_km_elec"], params["emission_PM10_g_km_elec"])
    ]:
        km_total = part * demande_eq * part_temps
        consommation_totale = km_total * (conso / 100) * surconso_congestion
        CO2_total = consommation_totale * CO2e / 1000 * jours
        NOx_total = km_total * NOx / 1_000_000 * jours
        PM10_total = km_total * PM / 1_000_000 * jours

        result["CO2"] += CO2_total
        result["NOx"] += NOx_total
        result["PM10"] += PM10_total

    return result

In [83]:
# --- Fusion des périodes pointe / hors-pointe ---
def emissions_horaires_combinees(params, vitesse_reduite=False, voie_covoit_activee=False):
    part_pointe = params["part_temps_pointe"]
    part_off = 1 - part_pointe

    surconso_pointe = params["surconso_congestion_pointe"]
    if voie_covoit_activee:
        surconso_pointe = min(1.05, surconso_pointe - 0.05)  # Réduction de congestion

    res_pointe = calcul_emissions(params, vitesse_reduite=vitesse_reduite, part_temps=part_pointe, surconso_congestion=surconso_pointe)
    res_off = calcul_emissions(params, vitesse_reduite=vitesse_reduite, part_temps=part_off, surconso_congestion=params["surconso_congestion_offpointe"])

    total = {k: res_pointe[k] + res_off[k] for k in res_pointe}
    return total

In [84]:
# --- Affichage des résultats ---
def afficher_resultats(params):
    scenarios = {
        "A. 70 km/h sans voie covoiturage": emissions_horaires_combinees(params, vitesse_reduite=False, voie_covoit_activee=False),
        "B. 50 km/h sans voie covoiturage": emissions_horaires_combinees(params, vitesse_reduite=True, voie_covoit_activee=False),
        "C. 50 km/h avec voie covoiturage": emissions_horaires_combinees(params, vitesse_reduite=True, voie_covoit_activee=True)
    }

    for nom, valeurs in scenarios.items():
        print(f"\n{nom} :")
        for polluant, val in valeurs.items():
            print(f"  {polluant} : {val:.2f} t/an")

    labels = list(scenarios.keys())
    CO2_vals = [scenarios[k]["CO2"] for k in labels]
    NOx_vals = [scenarios[k]["NOx"] for k in labels]
    PM10_vals = [scenarios[k]["PM10"] for k in labels]

    x = np.arange(len(labels))
    width = 0.25

    plt.figure(figsize=(10, 6))
    plt.bar(x - width, CO2_vals, width, label="CO2")
    plt.bar(x, NOx_vals, width, label="NOx")
    plt.bar(x + width, PM10_vals, width, label="PM10")
    plt.xticks(x, labels, rotation=15)
    plt.ylabel("t/an")
    plt.title("Émissions annuelles par scénario (voies horaires)")
    plt.legend()
    plt.tight_layout()
    plt.show()

In [85]:
# --- Interface interactive ---
@interact(
    distance_moyenne_km=FloatSlider(min=10, max=50, step=1, value=35, description="km/jour"),
    nombre_vehicules=IntSlider(min=50000, max=300000, step=10000, value=100000, description="véhicules/j"),
    part_covoiturage=FloatSlider(min=0.0, max=0.5, step=0.01, value=0.2, description="% covoiturage"),
    surconso_congestion_pointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.1, description="Surconso pointe"),
    surconso_congestion_offpointe=FloatSlider(min=1.0, max=1.2, step=0.01, value=1.0, description="Surconso hors-pointe")
)
def simulation_interactive(distance_moyenne_km, nombre_vehicules, part_covoiturage,
                           surconso_congestion_pointe, surconso_congestion_offpointe):
    params = default_params()
    params.update({
        "distance_moyenne_km": distance_moyenne_km,
        "nombre_vehicules": nombre_vehicules,
        "part_covoiturage": part_covoiturage,
        "surconso_congestion_pointe": surconso_congestion_pointe,
        "surconso_congestion_offpointe": surconso_congestion_offpointe
    })
    afficher_resultats(params)

interactive(children=(FloatSlider(value=35.0, description='km/jour', max=50.0, min=10.0, step=1.0), IntSlider(…